# Emotion Detection

Tester: Muhammad Aditya Bayhaqie

[Reference](https://paperswithcode.com/paper/facial-emotion-recognition-using-transfer)

## Library Preparation

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Connect with Drive Data

In [ ]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

## Import Datasets

In [ ]:
! mkdir ~/.kaggle

In [ ]:
!cp /content/drive/MyDrive/CollabData/kaggle_API/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle competitions download challenges-in-representation-learning-facial-expression-recognition-challenge

In [ ]:
!unzip challenges-in-representation-learning-facial-expression-recognition-challenge.zip

## Dataset Assesment

In [ ]:
data = pd.read_csv('icml_face_data.csv')

In [ ]:
data.shape()

In [ ]:
data.head()

### Image Test

In [ ]:
np_data = np.array(data)

In [ ]:
np_data.shape

In [ ]:
np_data[0]

In [ ]:
test_image = np_data[0][1].split(" ")
print(type(test_image))
print(len(test_image))